In [16]:
######################################################################
# Author: Alex Kim
# Date: 2024-04-26
#
# This script parses and converts encoded csv data into decoded SPSS
# file with variable labels derived from the encoded file.
#
# Usage:
# - Ensure the required libraries are installed in current R env
# - Update the 3 variables for input/output file paths
# - Run the script
######################################################################
suppressPackageStartupMessages({
    library(readr)
    library(dplyr)
    library(haven)
})
######################################################################
# User defined variables

data_dict_path <- './data/post-call/data_dict (1).csv'
encoded_data_path <- './data/post-call/data_encoded_04_27_2024 (1).csv'
spss_output_path <- './data/output/spss-output_04_27_2024.sav'

######################################################################

load_files <- function(encoded_data_path, data_dict_path) {
    data_dict <- read_csv(data_dict_path, show_col_types = FALSE)
    # Extract column names and descriptions from the first two rows before actually loading data
    tmp_data <- read_csv(encoded_data_path, col_names = FALSE, show_col_types = FALSE)
    col_names <- as.character(tmp_data[1, ])
    # Reload the data, skipping the first two rows to avoid the col description row
    data <- read_csv(encoded_data_path, skip = 2, col_names = col_names, show_col_types = FALSE)

    return(list(data = data, data_dict = data_dict))
}

# Important for adding variable labels for SPSS.
# The 2nd row in the encoded data files contain column descriptions.
get_col_descriptions <- function(encoded_data_path) {
    df <- read_csv(encoded_data_path, col_names = FALSE, show_col_types = FALSE)
    col_names <- as.character(df[1, ])
    col_descriptions <- as.character(df[2, ])
    col_descriptions <- setNames(col_descriptions, col_names)

    return(col_descriptions)
}

# Add variable labels as metadata for SPSS
add_var_labels <- function(data, encoded_data_path) {
    # Add variable labels for SPSS from the 2nd row of encoded data files
    col_descriptions <- get_col_descriptions(encoded_data_path)

    for(col in names(col_descriptions)) {
        if(col %in% names(data)) {
            attr(data[[col]], "label") <- col_descriptions[[col]]
        }
    }
    return (data)
}



# Parse the data dict for the encoded variable values
# and add as value labels for SPSS file.
get_value_labels <- function(data_dict) {
  # Filter out duplicate value-label pairs
  data_dict <- data_dict[!duplicated(paste(data_dict$variable, data_dict$label)), fromLast = TRUE]

   # Cast 'label' column to integers for nicer output in SPSS
  data_dict$label <- as.integer(data_dict$label)  
   
  # Transform the data dictionary into a list of value labels
  value_labels_list <- split(data_dict, data_dict$variable)
  
  # Transform into a named vector for each variable
  value_labels <- lapply(value_labels_list, function(x) {
    setNames(x$label, x$value)
  })
  
  return(value_labels)
}

# Function to add value labels to the data
add_value_labels <- function(data, data_dict_path) {
    value_labels <- get_value_labels(data_dict_path)
    for(var in names(value_labels)) {
        # Convert the list of labels to a named vector
        labels_vector <- value_labels[[var]]

        # Try casting to int here for pretty SPSS labels
        data[[var]] <- labelled(as.integer(data[[var]]), labels = labels_vector)    
    }
    return(data)
}

main <- function(encoded_dath_path, data_dict_path, spss_output_path) {
    # Import the encoded data and the data dict
    result <- load_files(encoded_data_path, data_dict_path)
    data <- result$data
    data_dict <- result$data_dict

    # Add factor/value labels for SPSS
    data <- add_value_labels(data, data_dict)
    
    # Add variable labels for SPSS
    data <- add_var_labels(data, encoded_data_path)
    
    # Export the processed data to an SPSS file
    write_sav(data, spss_output_path)
    print(paste("Data exported successfully to", spss_output_path))
    return (data)
}

df <- main(encoded_data_path, data_dict_path, spss_output_path)

[1] "Data exported successfully to ./data/output/spss-output_04_27_2024.sav"


In [ ]:
val_labels <- get_value_labels(data_dict)
val_labels

In [ ]:
# Function to check if all columns have labels
check_col_labels <- function(data) {
    for(col in names(data)) {
        label <- attr(data[[col]], "label")
        if(is.null(label)) {
            print(paste("Column:", col, "- Missing label"))
        } else {
            # print(paste("Column:", col, "- Label:", label))
        }
    }
}

library(haven)

# Function to check value labels assigned to variables
check_value_labels <- function(data) {
    for(col in names(data)) {
        if(is.labelled(data[[col]])) {
            labels <- attr(data[[col]], "labels")
            if(!is.null(labels)) {
                print(paste("Column:", col, "- Value Labels:"))
                for (key in names(labels)) {
                    print(paste("  ", key, "-", labels[key]))
                }
            } else {
                print(paste("Column:", col, "- No Value Labels Assigned"))
            }
        } else {
            print(paste("Column:", col, "- Not a labelled variable"))
        }
    }
}

# check_col_labels(df)
check_value_labels(df)

In [7]:
result = load_files(encoded_data_path, data_dict_path)
data = result$data
data_dict = result$data_dict

In [ ]:
# Removing dups from data_dict

dim(data_dict)
data_dict <- data_dict[!duplicated(paste(data_dict$variable, data_dict$label)), fromLast = TRUE]
dim(data_dict)

In [8]:
remove_duplicates <- function(df) {
  # Initialize an empty list to store indices of duplicate rows
  duplicates <- list()
  
  # Iterate through each row in the dataframe
  for (i in 1:nrow(df)) {
    # Check if the current combination of 'variable' and 'label' has been seen before
    if (any(df$variable[1:(i-1)] == df$variable[i] & df$label[1:(i-1)] == df$label[i])) {
      # Store the index of the duplicate row
      duplicates[[length(duplicates) + 1]] <- i
    }
  }
  
  # Remove duplicate rows from the dataframe
  df <- df[-unlist(duplicates), ]
  
  return(duplicates)
}

dim(data_dict)

x <- remove_duplicates(data_dict)

# dim(x)

[1] 397   3

In [12]:
remove_duplicates_from_data_dict <- function(df) {
  # Create a new column by concatenating 'variable' and 'label' values
  df$combined <- paste(df$variable, df$label, sep = "_")
  
  # Keep only the first occurrence of each unique combined value
  df <- df[!duplicated(df$combined), ]
  
  # Remove the combined column
  df <- df[, -which(names(df) == "combined")]
  
  return(df)
}

x <- remove_duplicates(data_dict)


dim(x)

[1] 385   3